In [1]:
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from ff import FF
from data import MNIST

In [2]:
batch_size_train = 32
batch_size_test = 32

In [3]:
train_loader = torch.utils.data.DataLoader(
  torchvision.datasets.MNIST('./datasets/MNIST/', train=True, download=True,
                             transform=torchvision.transforms.Compose([
                               torchvision.transforms.ToTensor(),
                               torchvision.transforms.Normalize(
                                 (0.1307,), (0.3081,))
                             ])),
  batch_size=batch_size_train, shuffle=True)

In [4]:
test_loader = torch.utils.data.DataLoader(
  torchvision.datasets.MNIST('./datasets/MNIST/', train=False, download=True,
                             transform=torchvision.transforms.Compose([
                               torchvision.transforms.ToTensor(),
                               torchvision.transforms.Normalize(
                                 (0.1307,), (0.3081,))
                             ])),
  batch_size=batch_size_test, shuffle=True)

In [5]:
config = {
    "in_dims": [784, 500, 500],
    "out_dims": [500, 500, 500],
    "epochs": 50,
    "threshold": 1.5,
}
num_layers = 3

In [6]:
model = FF(num_layers, config, torch.optim.Adam)

In [8]:
model.train()
for i, (x, y) in enumerate(train_loader):
    x_pos, _ = MNIST.overlay_y_on_x(x, y)
    rnd = torch.randperm(x.size(0))
    x_neg, _ = MNIST.overlay_y_on_x(x, y[rnd])
    #print(x_pos.requires_grad, x_neg.requires_grad)
    losses = model(x_pos, x_neg)
    print(losses)

0.8904549996058146
0.7119278844197591
0.6968376847108205
0.6945142245292665
0.6943762091795603
0.6944306111335754
0.6936002266407013
0.6934893743197122
0.6935726590951283
0.6935866951942443
0.693378122250239
0.693519124587377
0.6930455636978149
0.6933305589358012
0.6929041004180908
0.6933121689160665
0.6935051433245341
0.6932327262560527
0.6926033023993176
0.6925544202327728
0.6914953148365021
0.6903269414107006
0.6906576192378998
0.6894031997521718
0.687938203016917
0.6886534182230631
0.6881546227137249
0.6873592774073284
0.6856378269195557
0.6850971853733062
0.6850525907675425
0.6903017214934031
0.6832916494210561
0.6836666981379191
0.6782299606005351
0.6793849643071491
0.6884123639265697
0.6772475266456603
0.6765178402264912
0.6751298622290293
0.6768537716070812
0.6851858341693878
0.6735738968849182
0.6821114412943521
0.6769849793116252
0.6706606034437815
0.674903666973114
0.6665087751547495
0.6698934630552927
0.6700105202198028
0.6736619436740875
0.6550097382068635
0.67373182257016

KeyboardInterrupt: 

In [7]:
predictions, real = MNIST.predict(test_loader, model)
print("Accuracy ", np.sum(predictions == real)/len(real))

TypeError: super(type, obj): obj must be an instance or subtype of type